In [ ]:
import csv
import requests
import json

# Google Maps Places API endpoints
textsearch_api_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
placedetails_api_url = 'https://maps.googleapis.com/maps/api/place/details/json'

# Your Google Maps API key
api_key = 'Your API Key'

# List of areas to search for
areas = ['Areas of interest'] #['New York', 'London', 'Paris']

# Store the extracted event management company details
event_company_details = []

# Iterate through each area
for area in areas:
    # Send a request to the Places API for event management company search
    params = {
        'query': f'event management companies in {area}',
        'key': api_key
    } # change the "event management companies in" according to your prefferences.
    while True:
        response = requests.get(textsearch_api_url, params=params)
        data = json.loads(response.text)

        # Extract relevant information from the response
        for result in data['results']:
            place_id = result['place_id']

            # Send a request to the Place Details API using the place_id
            details_params = {
                'place_id': place_id,
                'fields': 'name,formatted_address,formatted_phone_number',
                'key': api_key
            }
            details_response = requests.get(placedetails_api_url, params=details_params)
            details_data = json.loads(details_response.text)

            # Extract event management company details from the Place Details API response
            details_result = details_data['result']
            name = details_result['name']
            address = details_result['formatted_address']
            phone_number = details_result.get('formatted_phone_number', '')

            # Store the event management company details
            event_company_details.append({
                'name': name,
                'address': address,
                'phone_number': phone_number
            })

        # Check if there is a next page of results
        next_page_token = data.get('next_page_token')
        if not next_page_token:
            break

        # Set the next page token for the next request
        params['pagetoken'] = next_page_token

# Store the extracted event management company details in a CSV file
csv_file = 'event_company_details.csv'

try:
    # Create and open the CSV file in write mode with UTF-8 encoding
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['name', 'address', 'phone_number'])
        writer.writeheader()

        # Write each event management company detail as a row in the CSV file
        for company in event_company_details:
            writer.writerow(company)
except Exception as e:
    print(f"Error occurred while writing to CSV: {str(e)}")
